# Simulador de Logística - LIA S.A.C.

Este notebook demuestra el uso del paquete `logistica_sim.sistema` para simular operaciones logísticas.

## Configuración Inicial

In [ ]:
import sys
sys.path.append('..')

from logistica_sim.sistema import GestionInventario, GestionTransporte
from logistica_sim.sistema.demanda import generar_demanda_diaria
from logistica_sim.sistema import catalogos
import pandas as pd
import main

---

# TESTS DEL SISTEMA

A continuación se presentan los tests principales que validan el correcto funcionamiento del sistema.

## Test 1: Validación de Imports de Módulos

**Qué valida:** Verifica que todos los módulos del sistema se importen correctamente y las clases principales se puedan instanciar sin errores.

In [ ]:
from logistica_sim.sistema import catalogos
print("✓ catalogos imported")

from logistica_sim.sistema import demanda
print("✓ demanda imported")

from logistica_sim.sistema import inventario
print("✓ inventario imported")

from logistica_sim.sistema import picking
print("✓ picking imported")

from logistica_sim.sistema import transporte
print("✓ transporte imported")

from logistica_sim.sistema import indicadores
print("✓ indicadores imported")

from logistica_sim.sistema import alertas
print("✓ alertas imported")

# Test inicialización de clases principales
gestion = inventario.GestionInventario()
print("✓ GestionInventario initialized")

gestion_trans = transporte.GestionTransporte()
print("✓ GestionTransporte initialized")

print("\n✅ ALL MODULE TESTS PASSED")

### Resultado del Test 1:

*[Pegar aquí captura de pantalla del resultado]*

---

## Test 2: Lógica de Stock Comprometido

**Qué valida:** Verifica que el sistema maneja correctamente el stock comprometido, nunca permite stock físico negativo, y gestiona adecuadamente situaciones de quiebre de stock (cuando la demanda excede el inventario disponible).

In [ ]:
print("=" * 80)
print("TEST 2: LÓGICA DE STOCK COMPROMETIDO")
print("=" * 80)

gestion = GestionInventario()

# Seleccionar un SKU para probar
sku_prueba = gestion.df_inventario.index[0]
stock_inicial = gestion.df_inventario.loc[sku_prueba, 'Stock_Fisico']
print(f"\nSKU de prueba: {sku_prueba}")
print(f"Stock Inicial: {stock_inicial}")

# CASO 1: Pedido Normal (Stock Suficiente)
print("\n" + "-" * 80)
print("CASO 1: Pedido Normal - Stock Suficiente")
print("-" * 80)

pedido_normal = {
    'id_pedido': 'P001',
    'cliente_id': 'C01',
    'zona_id': 'Z01',
    'items': [{'sku': sku_prueba, 'cantidad': 10}]
}

print(f"Solicitud: 10 unidades")
gestion.comprometer_stock(pedido_normal)
comprometido = gestion.df_inventario.loc[sku_prueba, 'Stock_Comprometido']
print(f"✓ Stock Comprometido: {comprometido} (esperado: 10)")

gestion.despachar_pedido(pedido_normal, 1)
comprometido_final = gestion.df_inventario.loc[sku_prueba, 'Stock_Comprometido']
fisico_final = gestion.df_inventario.loc[sku_prueba, 'Stock_Fisico']
print(f"✓ Tras despacho - Comprometido: {comprometido_final} (esperado: 0)")
print(f"✓ Tras despacho - Stock Físico: {fisico_final} (esperado: {stock_inicial - 10})")

# CASO 2: Pedido Excesivo (Quiebre de Stock)
print("\n" + "-" * 80)
print("CASO 2: Pedido Excesivo - Quiebre de Stock")
print("-" * 80)

# Forzar stock bajo para simular quiebre
gestion.df_inventario.loc[sku_prueba, 'Stock_Fisico'] = 5
gestion.df_inventario.loc[sku_prueba, 'Stock_Comprometido'] = 0
gestion._calcular_campos_derivados()

pedido_excesivo = {
    'id_pedido': 'P002',
    'cliente_id': 'C02',
    'zona_id': 'Z01',
    'items': [{'sku': sku_prueba, 'cantidad': 20}]
}

print(f"Stock disponible: 5 unidades")
print(f"Solicitud: 20 unidades")

gestion.comprometer_stock(pedido_excesivo)
comprometido = gestion.df_inventario.loc[sku_prueba, 'Stock_Comprometido']
print(f"✓ Stock Comprometido: {comprometido} (esperado: 5)")

items_despachados = gestion.despachar_pedido(pedido_excesivo, 1)
comprometido_final = gestion.df_inventario.loc[sku_prueba, 'Stock_Comprometido']
fisico_final = gestion.df_inventario.loc[sku_prueba, 'Stock_Fisico']
print(f"✓ Items despachados: {sum(i['cantidad'] for i in items_despachados)} (esperado: 5)")
print(f"✓ Tras despacho - Stock Físico: {fisico_final} (esperado: 0)")

print("\n" + "=" * 80)
if fisico_final >= 0:
    print("✅ TEST 2 EXITOSO: Stock comprometido funciona correctamente")
else:
    print("❌ TEST 2 FALLIDO")
print("=" * 80)

### Resultado del Test 2:

*[Pegar aquí captura de pantalla del resultado]*

---

## Test 3: Sistema de Compras y Kardex

**Qué valida:** Verifica que el sistema genera órdenes de compra automáticamente cuando el stock llega al punto de reorden, y que las recepciones se registran correctamente en el Kardex después del lead time.

In [ ]:
print("=" * 80)
print("TEST 3: SISTEMA DE COMPRAS Y KARDEX")
print("=" * 80)

print("\nEjecutando simulación de 15 días...")
resultados = main.run_simulation(n_dias=15, capacidad_picking=1500, escenario="normal")

df_kardex = resultados['df_kardex']
df_compras = resultados['df_compras']

print("\n" + "-" * 80)
print("RESULTADOS DE COMPRAS")
print("-" * 80)

total_compras = len(df_compras)
compras_recibidas = len(df_compras[df_compras['Estado'] == 'Recibido'])
compras_transito = total_compras - compras_recibidas

print(f"\nTotal de órdenes de compra generadas: {total_compras}")
print(f"✓ Órdenes recibidas: {compras_recibidas}")
print(f"✓ Órdenes en tránsito: {compras_transito}")

if not df_compras.empty:
    print("\n" + "-" * 80)
    print("PRIMERAS 5 ÓRDENES DE COMPRA")
    print("-" * 80)
    for idx, row in df_compras.head(5).iterrows():
        print(f"\n{row['ID_Compra']}: {row['Producto']} - Cantidad: {row['Cantidad']}")
        print(f"  Creado: Día {row['Fecha_Creacion']} → Arribo: Día {row['Fecha_Arribo']}")
        print(f"  Estado: {row['Estado']}")

print("\n" + "-" * 80)
print("MOVIMIENTOS EN KARDEX")
print("-" * 80)

compras_kardex = df_kardex[df_kardex['Tipo_Movimiento'] == 'COMPRA_RECEPCION']

if not compras_kardex.empty:
    print(f"\n✓ Total de recepciones en Kardex: {len(compras_kardex)}")
    print("\nPrimeras 5 recepciones:")
    for idx, row in compras_kardex.head(5).iterrows():
        print(f"Día {row['Fecha']:>2} | {row['Producto']:<8} | "
              f"Cantidad: {row['Cantidad']:>4} | Saldo: {row['Saldo_Final']:>5}")
else:
    print("⚠️  No se encontraron recepciones en el Kardex")

print("\n" + "=" * 80)
if total_compras > 0 and not compras_kardex.empty:
    print("✅ TEST 3 EXITOSO: Sistema de compras y Kardex funcionan correctamente")
else:
    print("❌ TEST 3 FALLIDO")
print("=" * 80)

### Resultado del Test 3:

*[Pegar aquí captura de pantalla del resultado]*

---

## Test 4: Validación de KPIs y Métricas

**Qué valida:** Verifica que los KPIs del sistema (OTIF, Fill Rate, Backlog Rate, Utilización de Flota) se calculan correctamente y están en rangos válidos (0-100%).

In [ ]:
print("=" * 80)
print("TEST 4: VALIDACIÓN DE KPIs Y MÉTRICAS")
print("=" * 80)

print("\nEjecutando simulación de 30 días...")
resultados = main.run_simulation(n_dias=30, capacidad_picking=1500, escenario="normal")

df_pedidos = resultados['df_pedidos']
metricas = resultados['metricas_globales']
resultados_diarios = resultados['resultados_diarios']

print("\n" + "-" * 80)
print("KPIs GLOBALES")
print("-" * 80)

# Calcular KPIs
total_pedidos = len(df_pedidos['ID_Pedido'].unique())
total_solicitado = df_pedidos['Cant_Solicitada'].sum()
total_entregado = df_pedidos['Cant_Entregada'].sum()

# OTIF
pedidos_agrupados = df_pedidos.groupby('ID_Pedido').agg({
    'Cant_Solicitada': 'sum',
    'Cant_Entregada': 'sum'
})
pedidos_perfectos = pedidos_agrupados[
    pedidos_agrupados['Cant_Solicitada'] == pedidos_agrupados['Cant_Entregada']
]
otif = (len(pedidos_perfectos) / total_pedidos * 100) if total_pedidos > 0 else 0

# Fill Rate
fill_rate = (total_entregado / total_solicitado * 100) if total_solicitado > 0 else 0

# Backlog Rate y Utilización
kpis_diarios = pd.DataFrame([d['kpis'] for d in resultados_diarios])
backlog_rate_avg = kpis_diarios['backlog_rate'].mean()
utilizacion_flota_avg = kpis_diarios['utilizacion_flota'].mean()

print(f"\n📊 Total de Pedidos: {total_pedidos}")
print(f"📦 Unidades Solicitadas: {total_solicitado:,.0f}")
print(f"📦 Unidades Entregadas: {total_entregado:,.0f}")
print(f"\n✓ OTIF: {otif:.1f}%")
print(f"✓ Fill Rate: {fill_rate:.1f}%")
print(f"✓ Backlog Rate: {backlog_rate_avg:.1f}%")
print(f"✓ Utilización Flota: {utilizacion_flota_avg:.1f}%")
print(f"✓ Valor Inventario Final: S/ {metricas['valor_total_inventario']:,.2f}")

# Validación
print("\n" + "-" * 80)
print("VALIDACIÓN DE RANGOS")
print("-" * 80)

tests_passed = 0
if 0 <= otif <= 100:
    print("✓ OTIF en rango válido (0-100%)")
    tests_passed += 1
if 0 <= fill_rate <= 100:
    print("✓ Fill Rate en rango válido (0-100%)")
    tests_passed += 1
if 0 <= backlog_rate_avg <= 100:
    print("✓ Backlog Rate en rango válido (0-100%)")
    tests_passed += 1
if 0 <= utilizacion_flota_avg <= 100:
    print("✓ Utilización Flota en rango válido (0-100%)")
    tests_passed += 1

print("\n" + "=" * 80)
if tests_passed == 4:
    print("✅ TEST 4 EXITOSO: Todos los KPIs están en rangos válidos")
else:
    print(f"⚠️  TEST 4 PARCIAL: {tests_passed}/4 validaciones pasadas")
print("=" * 80)

### Resultado del Test 4:

*[Pegar aquí captura de pantalla del resultado]*

---

## Test 5: Verificación Integral del Sistema

**Qué valida:** Ejecuta una verificación completa del sistema validando: stock no negativo, registro de stockouts, capacidad de transporte, y cálculo correcto de KPIs.

In [ ]:
print("=" * 80)
print("TEST 5: VERIFICACIÓN INTEGRAL DEL SISTEMA")
print("=" * 80)

print("\nEjecutando simulación de 10 días...")
res = main.run_simulation(n_dias=10, capacidad_picking=1500, escenario="normal")

# Test 1: Stock No Negativo
print("\n" + "-" * 80)
print("Test 1: Stock No Negativo")
print("-" * 80)
df_inv_final = res['resultados_diarios'][-1]['estado_inventario']
min_stock = df_inv_final['Stock_Fisico'].min()
print(f"Stock Físico Mínimo: {min_stock}")
if min_stock >= 0:
    print("✅ PASS: Stock Físico nunca es negativo")
else:
    print("❌ FAIL: Se encontró stock negativo")

# Test 2: Registro de Stockouts
print("\n" + "-" * 80)
print("Test 2: Registro de Stockouts")
print("-" * 80)
backlog = res.get('backlog', pd.DataFrame())
ventas_perdidas = res.get('ventas_perdidas', pd.DataFrame())
total_backlog = len(backlog) if not backlog.empty else 0
total_perdidas = len(ventas_perdidas) if not ventas_perdidas.empty else 0
print(f"Backlog: {total_backlog} items")
print(f"Ventas Perdidas: {total_perdidas} items")
if total_backlog > 0 or total_perdidas > 0:
    print("✅ PASS: Sistema registra situaciones de stockout")
else:
    print("ℹ️  INFO: No hubo stockouts (buen manejo de inventario)")

# Test 3: Capacidad de Transporte
print("\n" + "-" * 80)
print("Test 3: Capacidad de Transporte")
print("-" * 80)
df_despachos = res['df_despachos']
if not df_despachos.empty:
    overloaded = df_despachos[df_despachos['Peso_Total_Carga_kg'] > df_despachos['Capacidad_Max_kg']]
    if overloaded.empty:
        print(f"Total despachos: {len(df_despachos)}")
        print("✅ PASS: Todos los despachos dentro de capacidad")
    else:
        print(f"❌ FAIL: {len(overloaded)} despachos exceden capacidad")
else:
    print("⚠️  WARNING: No se generaron despachos")

# Test 4: KPIs
print("\n" + "-" * 80)
print("Test 4: Validación de KPIs")
print("-" * 80)
metricas = res['metricas_globales']
print(f"OTIF Global: {metricas.get('otif_global', 0):.1f}%")
print(f"Fill Rate Global: {metricas.get('fill_rate_global', 0):.1f}%")
if 0 <= metricas.get('otif_global', 0) <= 100:
    print("✅ PASS: KPIs en rangos válidos")
else:
    print("❌ FAIL: KPIs fuera de rango")

print("\n" + "=" * 80)
print("✅ VERIFICACIÓN INTEGRAL COMPLETADA")
print("=" * 80)

### Resultado del Test 5:

*[Pegar aquí captura de pantalla del resultado]*

---

## Resumen

Este notebook contiene 5 tests principales:

1. **Test 1:** Validación de imports de módulos
2. **Test 2:** Lógica de stock comprometido
3. **Test 3:** Sistema de compras y Kardex
4. **Test 4:** Validación de KPIs y métricas
5. **Test 5:** Verificación integral del sistema

Cada test valida aspectos críticos del sistema logístico.